## 数据预处理

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [2]:
# 读取数据
df = pd.read_csv('../data_new/ziguan_full.csv')

# 显示原始数据信息
print("原始数据信息:")
print(df.info())
print("\n原始数据样本:")
print(df.head())

原始数据信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54072 entries, 0 to 54071
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CUST_ID    54072 non-null  int64  
 1   CUST_TYPE  54072 non-null  int64  
 2   CONF_DATE  54072 non-null  int64  
 3   BUSI_CODE  54072 non-null  int64  
 4   FUND_CODE  54072 non-null  object 
 5   CONF_AMTS  54072 non-null  float64
 6   GENDER     26561 non-null  float64
 7   BIRTH      26561 non-null  float64
 8   TELL       46888 non-null  object 
 9   NET_CODE   47640 non-null  object 
 10  RISK_LEV   47640 non-null  float64
 11  COUNTY     26744 non-null  float64
 12  Matched    54072 non-null  bool   
dtypes: bool(1), float64(5), int64(4), object(3)
memory usage: 5.0+ MB
None

原始数据样本:
        CUST_ID  CUST_TYPE  CONF_DATE  BUSI_CODE FUND_CODE  CONF_AMTS  GENDER  \
0  369000460487          1   20240102        139    000596      50.00     2.0   
1  369000450364          1   2024010

In [3]:
# 2.1 处理缺失值
# 数值型列用0填充，类别型列用'missing'填充
num_cols = ['CONF_AMTS', 'GENDER', 'BIRTH', 'RISK_LEV', 'COUNTY','TELL']
cat_cols = ['CUST_TYPE', 'BUSI_CODE', 'FUND_CODE', 'NET_CODE']

df[num_cols] = df[num_cols].fillna(0)
df[cat_cols] = df[cat_cols].fillna('missing')

df.head()

,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,BIRTH,TELL,NET_CODE,RISK_LEV,COUNTY,Matched
0,369000460487,1,20240102,139,000596,50.00,2.0,19770726.0,139****9808,etrading,4.0,320102.0,False
1,369000450364,1,20240102,139,001986,10.00,1.0,19900106.0,138****8973,etrading,3.0,310107.0,False
2,369000610014,1,20240102,139,005928,1.00,2.0,19640306.0,136****5770,etrading,5.0,650104.0,False
3,369001110005,1,20240102,122,004399,95.77,2.0,19860711.0,186****7616,etrading,4.0,440305.0,False
4,369001110005,1,20240102,122,004399,23.94,2.0,19860711.0,186****7616,etrading,4.0,440305.0,False


In [4]:
# 2.2 处理日期特征
df['CONF_DATE'] = pd.to_datetime(df['CONF_DATE'], format='%Y%m%d')
df['CONF_YEAR'] = df['CONF_DATE'].dt.year
df['CONF_MONTH'] = df['CONF_DATE'].dt.month
df['CONF_DAY'] = df['CONF_DATE'].dt.day


df.head()

,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,BIRTH,TELL,NET_CODE,RISK_LEV,COUNTY,Matched,CONF_YEAR,CONF_MONTH,CONF_DAY
0,369000460487,1,2024-01-02,139,000596,50.00,2.0,19770726.0,139****9808,etrading,4.0,320102.0,False,2024,1,2
1,369000450364,1,2024-01-02,139,001986,10.00,1.0,19900106.0,138****8973,etrading,3.0,310107.0,False,2024,1,2
2,369000610014,1,2024-01-02,139,005928,1.00,2.0,19640306.0,136****5770,etrading,5.0,650104.0,False,2024,1,2
3,369001110005,1,2024-01-02,122,004399,95.77,2.0,19860711.0,186****7616,etrading,4.0,440305.0,False,2024,1,2
4,369001110005,1,2024-01-02,122,004399,23.94,2.0,19860711.0,186****7616,etrading,4.0,440305.0,False,2024,1,2


In [5]:
# 2.3 处理出生日期特征

def process_birth_data(df):
    """处理BIRTH字段的完整方案"""
    
    # 1. 类型转换与格式处理
    df['BIRTH'] = df['BIRTH'].astype('Int64').astype(str)  # 先转Int64避免精度丢失，再转字符串
    
    # 2. 有效日期过滤 (处理8位数字)
    mask_valid = df['BIRTH'].str.match(r'^\d{8}$') & ~df['BIRTH'].isin(['0', '99999999'])
    
    # 3. 日期转换
    df['BIRTH_DATE'] = pd.to_datetime(
        df.loc[mask_valid, 'BIRTH'],
        format='%Y%m%d',
        errors='coerce'
    )
    
    # 4. 计算年龄（当前年份 - 出生年份）
    current_year = pd.Timestamp.now().year
    df['AGE'] = (current_year - df['BIRTH_DATE'].dt.year)
    
    # 5. 年龄合理性校验
    age_mask = (df['AGE'] > 0) & (df['AGE'] < 120)
    df.loc[~age_mask, 'AGE'] = np.nan
    
    # 6. 缺失值处理（使用中位数填充）
    median_age = df['AGE'].median()
    df['AGE'] = df['AGE'].fillna(median_age).astype(int)
    
    
    # 8. 删除临时列
    df = df.drop(['BIRTH', 'BIRTH_DATE'], axis=1)
    
    # 9. 验证输出
    print("年龄分布统计:")
    print(df['AGE'].describe())

    
    return df

df = process_birth_data(df)
df.head()

年龄分布统计:
count    54072.000000
mean        43.496875
std          8.787953
min         20.000000
25%         42.000000
50%         42.000000
75%         42.000000
max         78.000000
Name: AGE, dtype: float64


,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,TELL,NET_CODE,RISK_LEV,COUNTY,Matched,CONF_YEAR,CONF_MONTH,CONF_DAY,AGE
0,369000460487,1,2024-01-02,139,000596,50.00,2.0,139****9808,etrading,4.0,320102.0,False,2024,1,2,48
1,369000450364,1,2024-01-02,139,001986,10.00,1.0,138****8973,etrading,3.0,310107.0,False,2024,1,2,35
2,369000610014,1,2024-01-02,139,005928,1.00,2.0,136****5770,etrading,5.0,650104.0,False,2024,1,2,61
3,369001110005,1,2024-01-02,122,004399,95.77,2.0,186****7616,etrading,4.0,440305.0,False,2024,1,2,39
4,369001110005,1,2024-01-02,122,004399,23.94,2.0,186****7616,etrading,4.0,440305.0,False,2024,1,2,39


In [6]:
# 2.4 处理电话号码特征
df['TELL_PREFIX'] = df['TELL'].apply(
    lambda x: f"{x[:3]}{x[-4:]}" if pd.notna(x) and len(str(x)) >= 7 else 0
)

df = df.drop('TELL', axis=1)
df.head()


,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,NET_CODE,RISK_LEV,COUNTY,Matched,CONF_YEAR,CONF_MONTH,CONF_DAY,AGE,TELL_PREFIX
0,369000460487,1,2024-01-02,139,000596,50.00,2.0,etrading,4.0,320102.0,False,2024,1,2,48,1399808
1,369000450364,1,2024-01-02,139,001986,10.00,1.0,etrading,3.0,310107.0,False,2024,1,2,35,1388973
2,369000610014,1,2024-01-02,139,005928,1.00,2.0,etrading,5.0,650104.0,False,2024,1,2,61,1365770
3,369001110005,1,2024-01-02,122,004399,95.77,2.0,etrading,4.0,440305.0,False,2024,1,2,39,1867616
4,369001110005,1,2024-01-02,122,004399,23.94,2.0,etrading,4.0,440305.0,False,2024,1,2,39,1867616


In [7]:
## 2.5. 改进的COUNTY处理（拆分320102.0为省、市、县）
def split_county(county_float):
    try:
        county_str = f"{int(county_float):06d}"
        return {
            'COUNTY_PROV': county_str[:2],
            'COUNTY_CITY': county_str[2:4],
            'COUNTY_DIST': county_str[4:6]
        }
    except:
        return {
            'COUNTY_PROV': '00',
            'COUNTY_CITY': '00',
            'COUNTY_DIST': '00'
        }

county_split = pd.DataFrame(df['COUNTY'].apply(split_county).tolist())
df = pd.concat([df.drop('COUNTY', axis=1), county_split], axis=1)

In [8]:
# 2.6 处理分类特征
# # 对高基数分类特征进行频数编码
# for col in ['FUND_CODE']:
#     freq = df[col].value_counts(normalize=True)
#     df[col+'_FREQ'] = df[col].map(freq)
#     df = df.drop(col, axis=1)

# 对低基数分类特征进行标签编码
le = LabelEncoder()
for col in ['NET_CODE', 'BUSI_CODE', 'CUST_TYPE']:
    df[col] = le.fit_transform(df[col])

df.head()



,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,NET_CODE,RISK_LEV,Matched,CONF_YEAR,CONF_MONTH,CONF_DAY,AGE,TELL_PREFIX,COUNTY_PROV,COUNTY_CITY,COUNTY_DIST
0,369000460487,1,2024-01-02,9,000596,50.00,2.0,1,4.0,False,2024,1,2,48,1399808,32,01,02
1,369000450364,1,2024-01-02,9,001986,10.00,1.0,1,3.0,False,2024,1,2,35,1388973,31,01,07
2,369000610014,1,2024-01-02,9,005928,1.00,2.0,1,5.0,False,2024,1,2,61,1365770,65,01,04
3,369001110005,1,2024-01-02,1,004399,95.77,2.0,1,4.0,False,2024,1,2,39,1867616,44,03,05
4,369001110005,1,2024-01-02,1,004399,23.94,2.0,1,4.0,False,2024,1,2,39,1867616,44,03,05


In [9]:
# 2.7 处理数值特征
# 归一化数值特征
scaler = MinMaxScaler()
scale_cols = ['AGE']
df[scale_cols] = scaler.fit_transform(df[scale_cols])


df.head()

,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,NET_CODE,RISK_LEV,Matched,CONF_YEAR,CONF_MONTH,CONF_DAY,AGE,TELL_PREFIX,COUNTY_PROV,COUNTY_CITY,COUNTY_DIST
0,369000460487,1,2024-01-02,9,000596,50.00,2.0,1,4.0,False,2024,1,2,0.482759,1399808,32,01,02
1,369000450364,1,2024-01-02,9,001986,10.00,1.0,1,3.0,False,2024,1,2,0.258621,1388973,31,01,07
2,369000610014,1,2024-01-02,9,005928,1.00,2.0,1,5.0,False,2024,1,2,0.706897,1365770,65,01,04
3,369001110005,1,2024-01-02,1,004399,95.77,2.0,1,4.0,False,2024,1,2,0.327586,1867616,44,03,05
4,369001110005,1,2024-01-02,1,004399,23.94,2.0,1,4.0,False,2024,1,2,0.327586,1867616,44,03,05


In [10]:
# 2.7 处理目标变量

df['target'] = df['Matched'].astype(int)
df = df.drop('Matched',axis=1)

In [11]:
null_counts = df.isnull().sum()

# 打印结果
print("每列的空值数量:")
print(null_counts)

每列的空值数量:
CUST_ID        0
CUST_TYPE      0
CONF_DATE      0
BUSI_CODE      0
FUND_CODE      0
CONF_AMTS      0
GENDER         0
NET_CODE       0
RISK_LEV       0
CONF_YEAR      0
CONF_MONTH     0
CONF_DAY       0
AGE            0
TELL_PREFIX    0
COUNTY_PROV    0
COUNTY_CITY    0
COUNTY_DIST    0
target         0
dtype: int64


In [12]:
# 5日金额

# 按客户ID和交易日期排序（确保时间顺序正确）
df = df.sort_values(by=['CUST_ID', 'CONF_DATE'])

# 计算每个客户每笔交易前5天的交易总额
df['5D_TOTAL'] = df.groupby('CUST_ID').apply(
    lambda group: group.rolling('5D', on='CONF_DATE')['CONF_AMTS'].sum()
).reset_index(level=0, drop=True)

# 减去当前行的金额（因为 rolling 包含当前行，但通常“5日内”指之前5天）
df['5D_TOTAL'] = df['5D_TOTAL'] - df['CONF_AMTS']
df['FLAG'] = 1

# 对每个客户检查前90天是否有交易
for cust_id, group in df.groupby('CUST_ID'):
    # 获取该客户的所有交易并按时间排序
    group = group.sort_values('CONF_DATE')
    for idx, row in group.iterrows():
        current_date = row['CONF_DATE']
        # 检查前90天是否有交易（不包括当前交易）
        prev_trans = group[group['CONF_DATE'] < current_date]
        prev_trans_in_90d = prev_trans[prev_trans['CONF_DATE'] >= (current_date - pd.Timedelta(days=90))]
        
        # 如果前90天无交易，则标记当前交易为10
        if prev_trans_in_90d.empty:
            df.loc[idx, 'FLAG'] = 10
            # 找到该客户之后3天的交易，并标记为10
            next_trans = group[
                (group['CONF_DATE'] > current_date) & 
                (group['CONF_DATE'] <= (current_date + pd.Timedelta(days=3)))
            ]
            df.loc[next_trans.index, 'FLAG'] = 10


df.head()

/tmp/ipykernel_351059/2284379991.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['5D_TOTAL'] = df.groupby('CUST_ID').apply(


,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,NET_CODE,RISK_LEV,CONF_YEAR,CONF_MONTH,CONF_DAY,AGE,TELL_PREFIX,COUNTY_PROV,COUNTY_CITY,COUNTY_DIST,target,5D_TOTAL,FLAG
8,18,1,2024-01-02,1,004399,0.6,2.0,1,5.0,2024,1,2,0.327586,1372592,31,01,04,0,0.0,10
221,18,1,2024-01-03,14,000917,0.0,2.0,1,5.0,2024,1,3,0.327586,1372592,31,01,04,0,0.6,10
222,18,1,2024-01-03,14,000917,0.0,2.0,1,5.0,2024,1,3,0.327586,1372592,31,01,04,0,0.6,10
223,18,1,2024-01-03,14,000917,0.0,2.0,1,5.0,2024,1,3,0.327586,1372592,31,01,04,0,0.6,10
225,18,1,2024-01-03,14,000917,0.0,2.0,1,5.0,2024,1,3,0.327586,1372592,31,01,04,0,0.6,10


In [13]:
user_sequences_df = {
    cust_id: group.drop('CUST_ID', axis=1) 
    for cust_id, group in df.groupby('CUST_ID')
}
# 查看某个用户的DataFrame序列
print(user_sequences_df[18].head(2))

     CUST_TYPE  CONF_DATE  BUSI_CODE FUND_CODE  CONF_AMTS  GENDER  NET_CODE  \
8            1 2024-01-02          1    004399        0.6     2.0         1   
221          1 2024-01-03         14    000917        0.0     2.0         1   

     RISK_LEV  CONF_YEAR  CONF_MONTH  CONF_DAY       AGE TELL_PREFIX  \
8         5.0       2024           1         2  0.327586     1372592   
221       5.0       2024           1         3  0.327586     1372592   

    COUNTY_PROV COUNTY_CITY COUNTY_DIST  target  5D_TOTAL  FLAG  
8            31          01          04       0       0.0    10  
221          31          01          04       0       0.6    10  


In [14]:
import pickle

with open('user_sequences.pkl', 'wb') as f:
    pickle.dump(user_sequences_df, f)  # 或保存 user_sequences_np

In [15]:


df = df.drop('CONF_DATE', axis=1)



In [16]:
# 保存为新的CSV文件
df.to_csv('../data_new/preprocessed_data_gnn_full.csv', index=False)

print("\n预处理完成！")


预处理完成！
